# Marginal Value Theorem
A place to test experimental parameters for foraging behavior in line with the marginal value theorem.

**NOTE: ipywidgets has poor compability with jupyterlab, particularly with versions >= 3.0. If interactive plots below do not display, try running this in a classic jupyter notebook instead.**

## Initial setup

### Imports

In [ ]:
# Numerical tools
import numpy as np
from scipy.optimize import broyden1

# Plotting tools
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
#%matplotlib widget
import matplotlib.pyplot as plt

# General tools
import copy
import warnings
warnings.filterwarnings('ignore', 'DeprecationWarning')

# Custom modules
import sys
sys.path.insert(0, '../python')
import helper

### Selection widget

In [ ]:
class SelectionSlider(widgets.SelectionSlider):
    
    def __init__(self, *args, return_index=True, transform=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.return_index = return_index
        if transform is not None:
            self.transform = transform
        else:
            self.transform = lambda x: x
        
    def get_interact_value(self):
        if self.return_index:
            return (self.transform(self.value), self.index)
        else:
            return self.transform(self.value)

In [ ]:
default_kwargs = dict(
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    return_index=True,
    transform=None
)

## Theory

### Optimal residence time
Let's define a patchy environment as consisting of the following parameters:

$\quad T_{p}^{(i)}$: time spent harvesting reward in patch $i$  
$\quad T_{t}^{(i)}$: time spent traveling to patch $i$  
$\quad R^{(i)}(t^{(i)})$: amount of reward harvested in patch $i$ after time $t^{(i)}$ (i.e. the *gain function*)  
$\quad s$: search cost per unit time

The average reward intake $\bar{E}$ across the environment is:

$
\begin{align}
\quad \bar{E} 
&= \frac{\text{total energy}}{\text{total time}} \\
&= \frac{\sum_{i} \left ( R^{(i)}(T_{p}^{(i)}) - sT_{t}^{(i)} \right )}{\sum_{i} \left ( T_{t}^{(i)} + T_{p}^{(i)} \right )} \\
&= \frac{R^{(i)}(T_{p}^{(i)}) - sT_{t}^{(i)} + k^{(i)}}{T_{t}^{(i)} + T_{p}^{(i)} + c^{(i)}}
\end{align}
$

where $k^{(i)} = \sum_{j \neq i} \left ( R^{(j)}(T_{p}^{(j)}) - sT_{t}^{(j)} \right )$ and $c^{(i)} = \sum_{j \neq i} \left ( T_{t}^{(j)} + T_{p}^{(j)} \right )$. Given that $\mathbf{T}_p = \{T_{p}^{(i)}\}$ are the only behavioral parameters in this model, we can define an optimal behavior vector as $\mathbf{T}_p^* = \{T_{p}^{*(i)}\}$. To solve for optimal behavior, we differentiate $\bar{E}$ with respect to $\mathbf{T}_p$:

$
\begin{align}
\quad \dfrac{\partial \bar{E}}{\partial \mathbf{T}_p} 
&= \left \{ \begin{matrix} ... & \dfrac{\partial \bar{E}}{\partial \mathbf{T}_p^{(i)}} & ... \end{matrix} \right \} \\
&= \left \{ \begin{matrix} ... & \dfrac{\partial}{\partial \mathbf{T}_p^{(i)}} \left ( \dfrac{R^{(i)}(T_{p}^{(i)}) - sT_{t}^{(i)} + k^{(i)}}{T_{t}^{(i)} + T_{p}^{(i)} + c^{(i)}} \right ) & ... \end{matrix} \right \} \\
&= \left \{ \begin{matrix} ... & \dfrac{r^{(i)}(T_{p}^{(i)}) \left ( T_{t}^{(i)} + T_{p}^{(i)} + c^{(i)} \right ) - \left ( R^{(i)}(T_{p}^{(i)}) - sT_{t}^{(i)} + k^{(i)} \right )}{\left ( T_{t}^{(i)} + T_{p}^{(i)} + c^{(i)} \right )^2} & ... \end{matrix} \right \}
\end{align}$


and note that $\bar{E}$ is maximized when $\frac{\partial \bar{E}}{\partial \mathbf{T}_p} = \mathbf{0}$:

$
\begin{align}
\quad \mathbf{0} &= \left \{ \begin{matrix} ... & \dfrac{r^{(i)}(T_{p}^{*(i)}) \left ( T_{t}^{(i)} + T_{p}^{*(i)} + c^{(i)} \right ) - \left ( R^{(i)}(T_{p}^{*(i)}) - sT_{t}^{(i)} + k^{(i)} \right )}{\left ( T_{t}^{(i)} + T_{p}^{*(i)} + c^{(i)} \right )^2} & ... \end{matrix} \right \} \\
&= \left \{ \begin{matrix} ... & r^{(i)}(T_{p}^{*(i)}) \left ( T_{t}^{(i)} + T_{p}^{*(i)} + c^{(i)} \right ) - \left ( R^{(i)}(T_{p}^{*(i)}) - sT_{t}^{(i)} + k^{(i)} \right ) & ... \end{matrix} \right \} \\
\end{align} \\
\quad \Rightarrow r^{(i)}(T_{p}^{*(i)}) \left ( T_{t}^{(i)} + T_{p}^{*(i)} + c^{(i)} \right ) = R^{(i)}(T_{p}^{*(i)}) - sT_{t}^{(i)} + k^{(i)} \\
\quad \Rightarrow r^{(i)}(T_{p}^{*(i)}) = \dfrac{R^{(i)}(T_{p}^{*(i)}) - sT_{t}^{(i)} + k^{(i)}}{T_{t}^{(i)} + T_{p}^{*(i)} + c^{(i)}} = \bar{E}
$

or in the single-patch case ($i=1$):

$
\quad r(T_{p}^{*}) = \dfrac{R(T_{p}^{*}) - sT_{t}}{T_{t} + T_{p}^{*}} = \bar{E}
$

where $r(t) = \frac{\mathrm{d} R}{\mathrm{d} t}$.

Therefore, the optimal patch residence time for each patch $T_{p}^{*(i)}$ occurs when the marginal gain in that patch equals the average rate of return across the environment; this is called the **marginal value theorem** (MVT). If we model the gain function based on an exponentially decaying rate of return within a given patch:

$
\quad r(T_{p}) = r_0 e^{-\frac{T_p}{\tau}} \\
\quad R(T_{p}) = \int_{0}^{T_p} r(t)dt = r_0 \tau \left ( 1 - e^{-\frac{T_p}{\tau}} \right ) + R_0
$

then, for the single-patch case, the MVT equation becomes:

$
\quad r(T_{p}^{*}) = \dfrac{R(T_{p}^{*}) - sT_{t}}{T_{t} + T_{p}^{*}} \\
\quad r_0 e^{-\frac{T_p^{*}}{\tau}} = \dfrac{r_0 \tau \left ( 1 - e^{-\frac{T_p^{*}}{\tau}} \right ) + R_0 - sT_{t}}{T_{t} + T_{p}^{*}} \\
\quad \Rightarrow r_0 e^{-\frac{T_p^{*}}{\tau}} \left ( T_{t} + T_{p}^{*} \right ) = r_0 \tau - r_0 \tau  e^{-\frac{T_p^{*}}{\tau}} + R_0 - sT_{t} \\
\quad \Rightarrow r_0 e^{-\frac{T_p^{*}}{\tau}} \left ( T_{t} + T_{p}^{*} + \tau \right ) - r_0 \tau - R_0 + sT_{t} = 0
$

As a sanity check, let's solve for the optimal residence time a slightly different way. If we are trying to maximize our average harvest rate across an environment, $\bar{R}(T_p)$, then we can simply set the derivative of this intake rate to zero and solve to find the maximum. First, the derivative is:

$
\quad \bar{R}(T_p) = \dfrac{R(T_{p}) - sT_{t}}{T_{t} + T_{p}} \\
\begin{align}
\quad \dfrac{d \bar{R}}{d T_p} 
&= \dfrac{\dfrac{d}{d T_p} \left ( R(T_p) - sT_t \right ) \left ( T_t + T_p \right ) - \left ( R(T_p) - sT_t \right ) \left ( \dfrac{d}{d T_p} \left ( T_t + T_p \right ) \right )}{\left ( T_t + T_p \right )^2} \quad \text{(by the product rule)} \\
&= \dfrac{\left ( \dfrac{d R}{d T_p} \right ) \left ( T_t + T_p \right ) - R(T_p) + sT_t}{\left ( T_t + T_p \right )^2} \\
&= \dfrac{r(T_p) \left ( T_t + T_p \right ) - R(T_p) + sT_t}{\left ( T_t + T_p \right )^2} \quad \text{(by definition)}
\end{align}
$

Setting this to zero, we get:

$
\begin{align}
\quad 0 
&= \dfrac{r(T^*_p) \left ( T_t + T^*_p \right ) - R(T^*_p) + sT_t}{\left ( T_t + T^*_p \right )^2} \\
&= r(T^*_p) \left ( T_t + T^*_p \right ) - R(T^*_p) + sT_t \\
&\Rightarrow r(T^*_p) = \dfrac{R(T^*_p) - sT_t}{T_t + T^*_p}
\end{align}
$

arriving at the same equation as above.

### Additional rewards and costs
Additional travel time can be modeled as beneficial rest.

## Parameter tool
Let's rewrite the code to 1) better display the data and 2) be a single code block to visualize any parameter manipulation.

There are five parameters ($T_P, T_T, R_0, r_0, \tau$), of which we can display three at a time using heatmaps. Because one of those, the unknown, is fixed, we have $C^4_2 = \frac{4!}{2! 2!} = 6$ graphs to display. Instead of displaying all graphs at once, we will instead allow the user to select which variable(s) to plot using either 1D (curve) or 2D (heatmap) visualization. Additionally, we will implement a selection tool for each fixed variable (i.e. not displayed) in order to quickly update the plots. 

In [ ]:
# Behavior parameters
#t_p = np.linspace(5.0, 60.0, num=56)
t_p = None
t_t = np.linspace(1.0, 30.0, num=30)

# Environment parameters
# Remember, R_0 / r_0 must be ≥ t_t, otherwise R_0 will default to zero!
R_0 = np.array([0.0])
r_0 = np.linspace(0.5, 5.0, num=19)
tau = np.linspace(1.0, 40.0, num=40)

# Parameters (do not change order!!!)
params = {'t_p': t_p,
          't_t': t_t,
          'R_0': R_0,
          'r_0': r_0,
          'tau': tau}
assert len([k for k, v in params.items() if v is None]) == 1

In [ ]:
X = {k: v for k, v in params.items() if v is not None}
name = [k for k, v in params.items() if v is None][0]
soln, R_opt, is_solvable = helper.get_optimal_values(**params, 
                                                     return_solvable=True, 
                                                     min_value=0.01, 
                                                     max_value=1000.0)
Y = {name: soln,
     'R_opt': R_opt}

In [ ]:
#%matplotlib widget # calling the magic function again closes previous figures

# Parameter settings
x_name = 'tau'

# Create widgets
w1D = {}
for name, x in X.items():
    if name != x_name:
        w1D[name] = SelectionSlider(options=x,
                                    value=x[0],
                                    description=name,
                                    **default_kwargs)

# Create initial plot
fig1D, ax1D = plt.subplots(1, 2, figsize=(8.0, 3.0))
idx = tuple([0 if name != x_name else slice(None) for name, x in X.items()])
lines = []
for i, (name, y) in enumerate(Y.items()):
    h, = ax1D[i].plot(X[x_name], y[idx], color='black', alpha=0.7)
    lines.append(h)
    ax1D[i].set_xlabel(x_name)
    ax1D[i].set_ylabel(name)
    ax1D[i].set_title('{} vs. {}'.format(name, x_name))
plt.tight_layout()
        
# Create update function
x_idx = list(X.keys()).index(x_name)
def update_plot(**kwargs):
    # Get indices to display
    idx = [val[1] for name, val in kwargs.items()] # val = (w.value, w.index)
    idx.insert(x_idx, slice(None)) # slice across x1 axis
    idx = tuple(idx)
    
    # Update curve
    for i, (name, y) in enumerate(Y.items()):
        lines[i].set_ydata(y[idx])
        ax1D[i].relim()
        ax1D[i].autoscale_view()
    fig1D.canvas.draw_idle()
    
widgets.interact(update_plot, **w1D);

In [ ]:
#%matplotlib widget # calling the magic function again closes previous figures

# Parameter settings
x1_name = 't_t'
x2_name = 'tau'
Y_range = [0.0, 1000.0]

# Create widgets
w2D = {} # different name from above because widgets persist in background
for name, x in X.items():
    if name not in [x1_name, x2_name]:
        w2D[name] = SelectionSlider(options=x,
                                    value=x[0],
                                    description=name,
                                    **default_kwargs)

# Create color palette
palette = copy.copy(plt.cm.coolwarm)
palette.set_under('black', Y_range[0])
palette.set_over('black', Y_range[1])
palette.set_bad(alpha=0.0)
        
# Determine axis labels 
# NOTE: in imshow(), first index is # rows, second index is # cols
x1_idx = list(X.keys()).index(x1_name)
x2_idx = list(X.keys()).index(x2_name)
if x1_idx < x2_idx:
    x1_label = x2_name
    x2_label = x1_name
else:
    x1_label = x1_name
    x2_label = x2_name
    
# Create initial heatmaps
fig2D, ax2D = plt.subplots(1, 2, figsize=(8.0, 3.0))
idx = tuple([0 if name not in [x1_name, x2_name] else slice(None) 
             for name, x in X.items()])
images = []
cbars = []
for i, (name, y) in enumerate(Y.items()):
    # Create heatmap
    extent = [X[x1_label][0], X[x1_label][-1], X[x2_label][0], X[x2_label][-1]]
    im = ax2D[i].imshow(y[idx],
                        cmap=palette,
                        aspect='auto',
                        origin='lower',
                        vmin=y[idx].min(),
                        vmax=y[idx].max(),
                        extent=extent)
    images.append(im)
    ax2D[i].set_xlabel(x1_label)
    ax2D[i].set_ylabel(x2_label)
    ax2D[i].set_title('{} vs. ({}, {})'.format(name, x1_name, x2_name))
    
    # Create colorbar
    cbar = fig2D.colorbar(im, ax=ax2D[i])
    cbar.set_label(name)
    cbars.append(cbar)
    
plt.tight_layout()

# Create update function
def update_heatmap(**kwargs):
    # Get indices to display
    idx = [val[1] for name, val in kwargs.items()] # val = (w.value, w.index)
    idx.insert(x1_idx, slice(None)) # slice across x1 axis
    idx.insert(x2_idx, slice(None)) # slice across x2 axis
    idx = tuple(idx)
    
    # Update heatmaps
    for i, (name, y) in enumerate(Y.items()):
        images[i].set_data(y[idx])
        cbars[i].mappable.set_clim(vmin=y[idx].min(), vmax=y[idx].max())
        ax2D[i].relim()
        ax2D[i].autoscale_view()
    fig2D.canvas.draw_idle()
    
widgets.interact(update_heatmap, **w2D);

## Parameter selection
Using the tool above, we want to select parameter sets that will maximize our behavioral readout (namely, residence time) given constraints of the animal, rig, etc. Thus, our goal is the following:

Find:

$\quad \{T_P^*, T_T, R_0, r_0, \tau\} \quad s.t. \quad \{T_T\} = argmax \left ( \dfrac{\partial T_P^*}{\partial T_T} \right ) \, , \, \{\tau\} = argmax \left ( \dfrac{\partial T_P^*}{\partial \tau} \right )$

given:

$
\begin{align}
\quad &5 \leq T_P^* \leq 40 \, seconds \quad &(1) \\
\quad &N_P^* \geq 20 \, in \, 30 \, mins \quad &(2) \\
\quad &R_t^* N_P^* \leq 600 \, uL \quad &(3) \\
\quad &\dfrac{R_t^*}{V_R} \geq 3 \quad &(4)
\end{align}
$

Let's quickly give justification for our constraints. The lower bound of (1) arises from needing a reasonable residence time to a) perform the task and b) accrue multiple rewards to allow for stochasticity to (eventually) play a role. The upper bound of (1) is an objective estimate of the maximum duration for which mice are comfortable poking, after which they unpoke due to an exploration drive, discomfort, etc. These numbers came from the residence times in the preliminary data: the lower bound from $\mu_{T_P} - \sigma_{T_P}$ and the upper bound from $\mu_{T_P} + \sigma_{T_P}$ in environments with the smallest and largest $T_P^*$, respectively. The constraint (2) comes from the observation in preliminary data that roughly 20 patches in 30 minutes is a good cutoff for distinguishing "good" from "bad" sessions. The constraint on total reward harvested in (3) assumes that after 20 patches of 5-6 five uL rewards, mice become less motivated (less hungry) to continue to engage in the foraging task. Lastly, the constraint (4) dictates that the number of harvested rewards at optimal leaving time must be at least 3, with the idea being that anything less a) is too easy to develop simple heuristics to solve, b) diminishes stochasticity introduced later on (i.e. there can only be so much variability of reward times for few rewards), and/or c) reduces the rate at which the animal is gaining information about the patch dynamics. From (3) and (4), it immediately follows that:

$
\quad 3 V_R \leq R_t^* \leq \dfrac{600 \, uL}{N_P^*} \quad (5)
$

Focusing on the maximization problem, we see that we want to find a set $\{T_T, \tau\}$ that maximizes the derivatives of the optimal residence time with respect to each parameter. Looking at our parameter tool, we can see that for both parameters, the derivative is maximized at the smallest possible value; in other words, the second derivatives appear to be monotonically decreasing. This suggests that we should simply operate on the extrema within our allowed parameter space given our constraints; that is, choose the smallest and largest values of $T_T$ and $\tau$ that still operate within our constrained space. 

There's a couple of approaches to tackle this problem, each with its own underlying assumptions:
1. Jointly optimize $\{T_T, \tau\}$ by maximizing $\left \| \nabla T_P^* \right \| = \left \| \left \langle \dfrac{\partial T_P^*}{\partial T_T}, \dfrac{\partial T_P^*}{\partial \tau} \right \rangle \right \| = \sqrt{\left ( \dfrac{\partial T_P^*}{\partial T_T} \right )^2 + \left ( \dfrac{\partial T_P^*}{\partial \tau} \right )^2}$. Choose the starting point to be the smallest allowed travel time and decay rate values (which maximizes the partial derivatives), and then follow a path based on gradient ascent until the maximum possible travel time is reached.
2. Fix $T_T$ to the average travel time for an animal on a fixed track length. Choose $\tau$ that maximize the change in $T_P^*$ holding either $T_T$ fixed (decay rate manipulation) or $\tau$ fixed (travel time manipulation).

In the first approach, we model the foraging environment as a more continuous space in which the perceived travel time and decay rate may move more fluidly despite fixed track lengths and true decay rates. In this case, it may be best to stay on a path in which the gradient is maximized so that for any change in the perceived parameters, the change in optimal residence time (our primary behavioral readout!) is maximized. The higher degree of stochasticity in the reward times, the more this approach becomes favorable.



In the second approach, we assume a more deterministic environment in which travel time is highly correlated with track length and (perceived) decay rate is relatively stable and easy to learn. In this case, we can assume that $T_T$ is already predetermined by the combination of track length and animal behavior (speed, engagement, etc.), so we can simply find the average travel time for the one and four meter tracks. (Note that the choice of extrema was already in our track design anyways: we made the smallest and largest tracks possible.) Moreover, $R_0$ is zero by design, so we can ignore that parameter for our calculations. That leaves us with just $\tau$ and $r_0$.

Working in our favor is the independence of $r_0$ from $T_P^*$. The initial reward rate affects the reward harvested for a given task but does not influence the optimal leaving time for the task, regardless of the point in parameter space. Thus we can treat our selection of $\tau$ and $r_0$ independently: $\tau$ will influence our first two constraints, $r_0$ our last constraint (we will choose $\tau$ first so that $N_P^*$ becomes a known quantity.). 

First, let's focus on $\tau$. For the one-meter linear track, the travel times were approximately 4-5 seconds, whereas preliminary data for the three-meter S-track were 11-12 seconds. We'll assume that the four-meter L-track will have travel times of 15 seconds. In the first case, 

Current thoughts:
- Finsh out gradient approach by plotting, side-by-side: dTp/dTt, dTp/dtau, ||del T_p||, direction of del T_p (arrow diagram?)
- Looking at the initial gradient diagrams, there's really no middle ground for maximizing ||del T_p||; you either choose mostly maximizing dT_p/dTt (which occurs at low Tt, high tau) or mostly maximizing dT_p/dtau (which occurs at high Tt, low tau).
- Maybe this leads us to choose two tau values, one that maximizes dT_p/dTt (which will be a high tau value) and one that maximizes dT_p/dtau (which will be a low tau value). These tau will be constrained by the limitations above, so essentially we'll be choosing the lowest and highest possible allowed values of tau (number of rewards, perceptable decay, etc.). Then, when combined with the travel times that are fixed by the track lengths, we have four possible combinations. From these four data points, we will lastly choose our r_0 value based on the reward constraints.
- From the behavior data analysis, it appears that:
    1. Travel time should be between 5.5 and 43 seconds.
    2. The maximum accrued reward over a session should be 400 uL.

In [ ]:
# Cache current slice of solutions for given (r0, R0)
# Because T_P is independent of both r0 and R0, any slice
# will give the same values and is valid for our analysis below.
assert t_p is None # analysis assumes we solved for T_P
idx = tuple([0 if name not in [x1_name, x2_name] else slice(None) 
             for name, x in X.items()])
y = Y['t_p'][idx]

In [ ]:
# First, plot ratio of T_P to tau to assess permissible values
fig, ax = plt.subplots()
cmap = plt.get_cmap('coolwarm')
thresh = [1.0, -np.log(0.5)]

# Plot boundaries
x1 = X[x1_label] # tau
x2 = X[x2_label] # t_t
for thresh_ in thresh:
    # Estimate boundary in (x1, x2) space where ratio crosses threshold
    idx = np.argwhere(np.logical_and(y[:-1, :] / x1[np.newaxis, :] <= thresh_,
                                     y[1:, :] / x1[np.newaxis, :] > thresh_))
    A = np.vstack([x1[idx[:, 1]], np.ones(idx.shape[0])]).T
    (m, b) = np.linalg.lstsq(A, x2[idx[:, 0]], rcond=None)[0]

    # Plot estimated boundary
    ax.plot(x1[idx[:,1]],
            m*x1[idx[:,1]] + b,
            color='black',
            linestyle='--',
            label=r'$\dfrac{T_P}{\tau}$' + ' = {:.2f}'.format(thresh_))

# Plot heatmap of ratio values
img = ax.imshow(y/x1[np.newaxis, :], 
                cmap=cmap, 
                vmin=0.0,
                vmax=2.0,  
                origin='lower', 
                extent=extent)
cb = fig.colorbar(img, ax=ax, ticks=[0.0, 0.5, 1.0, 1.5, 2.0])

# Label plot
ax.set_title('estimated boundary for leaving time')
ax.set_xlabel(x1_label)
ax.set_ylabel(x2_label)
cb.ax.set_yticklabels(['0.0', '0.5', '1.0', '1.5', '2.00+'])
cb.ax.set_ylabel(r'$\dfrac{T_P}{\tau}$')
ax.legend()

plt.tight_layout();

In [ ]:
# Plot gradient wrt travel time and decay rate
fig, ax = plt.subplots(2, 2, figsize=(10, 8))
cmap = plt.get_cmap('BrBG') # different color scheme to avoid confusion

# Estimate partial derivatives and gradient
dydtt = np.diff(y, axis=0)
dydtau = np.diff(y, axis=1)
grad = np.sqrt(dydtt[:, 1:]**2 + dydtau[1:, :]**2)

# Plot partial derivatives
img = ax[0, 0].imshow(dydtt, 
                      cmap=cmap,
                      origin='lower',  
                      #vmin=grad.min(),
                      #vmax=grad.max(),
                      extent=extent)
fig.colorbar(img, ax=ax[0, 0], shrink=0.6)
ax[0, 0].set_title(r'$\dfrac{\partial T_P}{\partial T_T}$')
img = ax[0, 1].imshow(dydtau, 
                      cmap=cmap,
                      origin='lower',
                      #vmin=grad.min(),
                      #vmax=grad.max(),
                      extent=extent)
cbar = fig.colorbar(img, ax=ax[0, 1], shrink=0.6)
ax[0, 1].set_title(r'$\dfrac{\partial T_P}{\partial \tau}$')

# Plot gradient magnitude
img = ax[1, 0].imshow(grad,
                      cmap=cmap,
                      origin='lower',
                      vmin=grad.min(),
                      vmax=grad.max(),
                      extent=extent)
cbar = fig.colorbar(img, ax=ax[1, 0], shrink=0.6)
ax[1, 0].set_title(r'$\left \|\| \nabla T_P^* \right \|\|$')

# Plot gradient direction overlaid on contour map
ax[1, 1].contour(y, 
                 cmap=palette, 
                 origin='lower', 
                 extent=extent)
ax[1, 1].quiver(X['tau'][1:], X['t_t'][1:], 
                dydtau[1:, :],dydtt[:, 1:], grad, 
                cmap=cmap, 
                angles='xy')
ax[1, 1].set_title('contour and quiver plot')

# Label axes
for ax_ in ax.reshape(-1):
    ax_.set_xlabel(x1_label)
    ax_.set_ylabel(x2_label)

plt.tight_layout();

(left, bottom, w_old, h_old) = ax[1, 1].get_position().bounds
(_, _, w_new, h_new) = ax[1, 0].get_position().bounds
ax[1, 1].set_position((left + (w_old - w_new)/2, 
                       bottom + (h_old - h_new)/2,
                       w_new, 
                       h_new));

plt.savefig('/home/james/Desktop/mvt_params.jpg', dpi=300, fmt='jpg')

Let's digest the above plots. First, looking at the boundary condition where $\frac{T_P^*}{\tau} \geq 1$, we see that the corner of the plot with small $T_T$ and large $\tau$ may be problematic, since the area in $(T_T, \tau)$ space defined by $\frac{T_P^*}{\tau} < 1$ corresponds to a less perceptible decay rate (the reward rate will still be greater than $\frac{r_0}{e}$ upon optimal leaving).

In [ ]:
# Next, plot minimum and maximum r_0 within constraints for each point (T_T, tau)
v_r = 2.0 # volume per reward (uL)
n_r_min = 3 # minimum number of discrete rewards per patch at optimal leaving time
R_max = 400 # maximum amount of reward (uL) in session

r0_min = lambda t_p, tau: (n_r_min*v_r)/(tau*(1 - np.exp(-t_p/tau)))
r0_max = lambda t_p, tau, t_t: R_max/((1800/(t_p + t_t))*tau*(1 - np.exp(-t_p/tau)))

fig, ax = plt.subplots(1, 2, figsize=(10, 5))

img = ax[0].imshow(r0_min(y, tau[np.newaxis, :]),
                      cmap=cmap,
                      origin='lower',  
                      #vmin=grad.min(),
                      #vmax=grad.max(),
                      extent=extent)
fig.colorbar(img, ax=ax[0], shrink=0.6)

img = ax[1].imshow(r0_max(y, tau[np.newaxis, :], t_t[:, np.newaxis]),
                      cmap=cmap,
                      origin='lower',  
                      #vmin=grad.min(),
                      #vmax=grad.max(),
                      extent=extent)
fig.colorbar(img, ax=ax[1], shrink=0.6);

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

ax.plot_surface(tau[np.newaxis, :],
                t_t[:, np.newaxis],
                r0_min(y, tau[np.newaxis, :]),
                cmap=cmap,
                alpha=0.5,
                linewidths=0.7,
                edgecolors='gray')

Z = r0_max(y, tau[np.newaxis, :], t_t[:, np.newaxis])
ax.plot_surface(tau[np.newaxis, :],
                t_t[:, np.newaxis],
                r0_max(y, tau[np.newaxis, :], t_t[:, np.newaxis]),
                cmap=cmap,
                alpha=0.5,
                linewidths=0.7,
                edgecolors='black')
cset = ax.contour(tau[np.newaxis, :]*np.ones(Z.shape),
                  t_t[:, np.newaxis]*np.ones(Z.shape),
                  r0_max(y, tau[np.newaxis, :], t_t[:, np.newaxis]), 
                  zdir='x', 
                  offset=-5, 
                  cmap=cmap)
ax.set_xlim([0.0, 40.0]);

## Archive

### Fit environment to behavior
Given preferred patch residence and travel times, what is the environment in which such behavior is optimal?

In [ ]:
# Behavior parameters (fixed)
t_p = 20
t_t = 10

# Environment parameters (varied)
# Remember, R_0 / r_0 must be ≥ t_t, otherwise R_0 will default to zero!
R_0 = np.array([0.0, 2.0, 6.0, 8.0, 10.0, 12.0])
r_0 = np.arange(50, 501, 25)/100

# Solve for tau, optimum cumulative reward
tau = np.zeros([R_0.shape[0], r_0.shape[0]])
R_opt = np.zeros([R_0.shape[0], r_0.shape[0]])
is_solvable = np.ones([R_0.shape[0], r_0.shape[0]], dtype=np.bool)
for i, R_0_ in enumerate(R_0):
    tau[i, :], R_opt[i, :], is_solvable[i, :] = get_optimal_values(t_p=t_p, 
                                                                   t_t=t_t, 
                                                                   R_0=R_0_, 
                                                                   r_0=r_0, 
                                                                   return_solvable=True)

In [ ]:
fig, ax = plt.subplots(tau.shape[0], 2, figsize=(10, 4*tau.shape[0]))

for i in range(tau.shape[0]):
    ax[i, 0].plot(r_0[is_solvable[i]], tau[i, is_solvable[i]], linestyle='-', color='gray')
    ax[i, 0].plot(r_0[np.invert(is_solvable[i])], tau[i, np.invert(is_solvable[i])], linestyle='--', color='gray') 
    ax[i, 0].set_xlabel('r_0 (uL/s)')
    ax[i, 0].set_ylabel('tau (s)')
    ax[i, 0].set_title('t_p=%d, t_t=%d, R_0=%d' % (t_p, t_t, R_0[i]))
    ax[i, 0].set_yscale('log')
    
    ax[i, 1].plot(r_0[is_solvable[i]], R_opt[i, is_solvable[i]], linestyle='-', color='gray')
    ax[i, 1].plot(r_0[np.invert(is_solvable[i])], R_opt[i, np.invert(is_solvable[i])], linestyle='--', color='gray')
    ax[i, 1].set_xlabel('r_0 (uL/s)')
    ax[i, 1].set_ylabel('R_opt (uL)')
    ax[i, 1].set_title('t_p=%d, t_t=%d, R_0=%d' % (t_p, t_t, R_0[i]))

ax[0, 0].set_ylim([1.0, 100.0])    
plt.tight_layout()

### Fit behavior to environment
Given environmental parameters, what is optimum behavior?

In [ ]:
# Behavior parameters
t_t = 10

# Environment parameters
# Remember, R_0 / r_0 must be ≥ t_t, otherwise R_0 will default to zero!
R_0 = np.array([0.0])
r_0 = np.arange(50, 501, 50)/100
tau = np.arange(10, 60, 5)

# Solve for residence time, optimum cumulative reward
t_p = np.zeros([tau.shape[0], r_0.shape[0]])
R_opt = np.zeros([tau.shape[0], r_0.shape[0]])
is_solvable = np.ones([tau.shape[0], r_0.shape[0]], dtype=np.bool)
for i, tau_ in enumerate(tau):
    t_p[i, :], R_opt[i, :], is_solvable[i, :] = get_optimal_values(t_t=t_t, 
                                                                   R_0=R_0, 
                                                                   r_0=r_0,
                                                                   tau=tau_,
                                                                   return_solvable=True)

In [ ]:
fig, ax = plt.subplots(t_p.shape[0], 2, figsize=(10, 20))

for i in range(t_p.shape[0]):
    ax[i, 0].plot(r_0[is_solvable[i]], t_p[i, is_solvable[i]], linestyle='-', color='gray')
    ax[i, 0].plot(r_0[np.invert(is_solvable[i])], t_p[i, np.invert(is_solvable[i])], linestyle='--', color='gray') 
    ax[i, 0].set_xlabel('r_0 (uL/s)')
    ax[i, 0].set_ylabel('t_p (s)')
    ax[i, 0].set_title('tau=%d, t_t=%d, R_0=%d' % (tau[i], t_t, R_0))
    
    ax[i, 1].plot(r_0[is_solvable[i]], R_opt[i, is_solvable[i]], linestyle='-', color='gray')
    ax[i, 1].plot(r_0[np.invert(is_solvable[i])], R_opt[i, np.invert(is_solvable[i])], linestyle='--', color='gray')
    ax[i, 1].set_xlabel('r_0 (uL/s)')
    ax[i, 1].set_ylabel('R_opt (uL)')
    ax[i, 1].set_title('tau=%d, t_t=%d, R_0=%d' % (tau[i], t_t, R_0))

#ax[0, 0].set_ylim([0.0, 100.0])    
plt.tight_layout()

In [ ]:
# vary travel time and solve for residence time; see what values double residence time
# Behavior parameters
t_t = np.arange(5, 41, 5)

# Environment parameters
# Remember, R_0 / r_0 must be ≥ t_t, otherwise R_0 will default to zero!
R_0 = np.array([0.0])
r_0 = np.arange(50, 501, 50)/100
tau = np.array([24.0])

# Solve for residence time, optimum cumulative reward
t_p = np.zeros([t_t.shape[0], r_0.shape[0]])
R_opt = np.zeros([t_t.shape[0], r_0.shape[0]])
is_solvable = np.ones([t_t.shape[0], r_0.shape[0]], dtype=np.bool)
for i, t_t_ in enumerate(t_t):
    t_p[i, :], R_opt[i, :], is_solvable[i, :] = get_optimal_values(t_t=t_t_, 
                                                                   R_0=R_0, 
                                                                   r_0=r_0,
                                                                   tau=tau,
                                                                   return_solvable=True)

In [ ]:
fig, ax = plt.subplots(t_p.shape[0], 2, figsize=(10, 20))

for i in range(t_p.shape[0]):
    ax[i, 0].plot(r_0[is_solvable[i]], t_p[i, is_solvable[i]], linestyle='-', color='gray')
    ax[i, 0].plot(r_0[np.invert(is_solvable[i])], t_p[i, np.invert(is_solvable[i])], linestyle='--', color='gray') 
    ax[i, 0].set_xlabel('r_0 (uL/s)')
    ax[i, 0].set_ylabel('t_p (s)')
    ax[i, 0].set_title('tau=%d, t_t=%d, R_0=%d' % (tau, t_t[i], R_0))
    
    ax[i, 1].plot(r_0[is_solvable[i]], R_opt[i, is_solvable[i]], linestyle='-', color='gray')
    ax[i, 1].plot(r_0[np.invert(is_solvable[i])], R_opt[i, np.invert(is_solvable[i])], linestyle='--', color='gray')
    ax[i, 1].set_xlabel('r_0 (uL/s)')
    ax[i, 1].set_ylabel('R_opt (uL)')
    ax[i, 1].set_title('tau=%d, t_t=%d, R_0=%d' % (tau, t_t[i], R_0))

#ax[0, 0].set_ylim([0.0, 100.0])    
plt.tight_layout()

#### How does optimal harvest rate change with different environments?

In [ ]:
# Behavior parameters
t_t = 5

# Environment parameters
# Remember, R_0 / r_0 must be ≥ t_t, otherwise R_0 will default to zero!
R_0 = np.array([0.0])
r_0 = np.arange(50, 501, 50)/100
tau = np.arange(10, 30, 1)

# Solve for residence time, optimum cumulative reward
t_p = np.zeros([r_0.shape[0], tau.shape[0]])
R_opt = np.zeros([r_0.shape[0], tau.shape[0]])
is_solvable = np.ones([r_0.shape[0], tau.shape[0]], dtype=np.bool)
for i, r_0_ in enumerate(r_0):
    t_p[i, :], R_opt[i, :], is_solvable[i, :] = get_optimal_values(t_t=t_t, 
                                                                   R_0=R_0, 
                                                                   r_0=r_0_,
                                                                   tau=tau,
                                                                   return_solvable=True)

In [ ]:
fig, ax = plt.subplots(r_0.shape[0], 2, figsize=(10, 20))

for i in range(r_0.shape[0]):
    ax[i, 0].plot(tau[is_solvable[i]], t_p[i, is_solvable[i]], linestyle='-', color='gray')
    ax[i, 0].plot(tau[np.invert(is_solvable[i])], t_p[i, np.invert(is_solvable[i])], linestyle='--', color='gray') 
    ax[i, 0].set_xlabel('tau (uL/s)')
    ax[i, 0].set_ylabel('t_p (s)')
    ax[i, 0].set_title('r_0=%.2f, t_t=%d, R_0=%d' % (r_0[i], t_t, R_0))
    
    ax[i, 1].plot(tau[is_solvable[i]], R_opt[i, is_solvable[i]]/(t_t + t_p[i, is_solvable[i]]), 
                  linestyle='-', color='gray')
    ax[i, 1].plot(tau[np.invert(is_solvable[i])], R_opt[i, np.invert(is_solvable[i])]/(t_t + t_p[i, np.invert(is_solvable[i])]), 
                  linestyle='--', color='gray') 
    ax[i, 1].set_xlabel('tau (uL/s)')
    ax[i, 1].set_ylabel('hr (uL/s)')
    ax[i, 1].set_title('r_0=%.2f, t_t=%d, R_0=%d' % (r_0[i], t_t, R_0))

plt.tight_layout()

In [ ]:
t_t_ = 10
R_0_ = R_0[0]
r_0_ = 2.0
t_p_ = t_p[np.argwhere(t_t == t_t_)[0], np.argwhere(r_0 == r_0_)[0]] 
tau_ = tau[0]

print('t_p = %.2f, t_t = %.2f, R_0 = %.2f, r_0 = %.2f, tau = %.2f' % (t_p_, t_t_, R_0_, r_0_, tau_))

In [ ]:
t_t_ = 25
R_0_ = R_0[0]
r_0_ = 2.0
t_p_ = t_p[np.argwhere(t_t == t_t_)[0], np.argwhere(r_0 == r_0_)[0]] 
tau_ = tau[0]

print('t_p = %.2f, t_t = %.2f, R_0 = %.2f, r_0 = %.2f, tau = %.2f' % (t_p_, t_t_, R_0_, r_0_, tau_))

### Fixed plotting tool

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(2, 6, figsize=(15, 5))

k = 0
Y_range = [0.0, 50.0]

# Create custom colormap
# (see https://matplotlib.org/3.1.0/gallery/images_contours_and_fields/image_masked.html)
palette = copy.copy(plt.cm.coolwarm)
palette.set_under('black', Y_range[0])
palette.set_over('black', Y_range[1])
palette.set_bad(alpha=0.0)
for i, (name, x) in enumerate(X.items()):
    # Plot parameter vs. unknown
    if isinstance(x, np.ndarray):
        if x.size > 1:
            ax[0, i].plot(x, helper.random_slice(Y, axis=i))
            ax[0, i].set_xlabel(x_name)
            ax[0, i].set_ylabel(Y_name)
            #ax[0, i].set_title(', '.join(['{}={}'.format(name, val) for name, val in ])
        else:
            ax[0, i].axis('off')
    else:
        ax[0, i].axis('off')
    
    # Plot heatmap with other parameters
    for j, (x_name_, x_) in enumerate(zip(X_name, X)):
        if j > i:
            h = ax[1, k].imshow(helper.random_slice(Y, axis=(i, j)),
                                cmap=palette,
                                aspect='auto',
                                origin='lower',
                                vmin=Y_range[0],
                                vmax=Y_range[1])
            ax[1, k].set_xlabel(x_name_)
            ax[1, k].set_ylabel(x_name)
            k += 1
            
fig.colorbar(h, ax=ax[1, k-1])
plt.tight_layout()